In [3]:
import os, sys, os.path
from ftplib import FTP
import xml.etree.ElementTree as ET
import csv
import gzip
import os
import glob
import sys
#reload(sys)

#Encode foreign names in utf-8
#sys.setdefaultencoding('utf-8')

In [20]:
#Data accessed in December 2017
#Downloading files from medline baseline
ftp=FTP("ftp.ncbi.nlm.nih.gov")
ftp.login()
ftp.cwd('pubmed')
ftp.cwd('baseline')
filenames=ftp.nlst()
#File #533 is the first file with publication date in 2005
goodFiles=[file for file in filenames if (file[-1]=="z" and int(file[9:13])>=533)]
for filename in goodFiles:
    print 'writing {}'.format(filename) 
    ftp.retrbinary("RETR " + filename ,open(filename, 'wb').write)
ftp.close()

writing pubmed18n0813.xml.gz
writing pubmed18n0814.xml.gz
writing pubmed18n0815.xml.gz
writing pubmed18n0816.xml.gz
writing pubmed18n0817.xml.gz
writing pubmed18n0818.xml.gz
writing pubmed18n0819.xml.gz
writing pubmed18n0820.xml.gz
writing pubmed18n0821.xml.gz
writing pubmed18n0822.xml.gz
writing pubmed18n0823.xml.gz
writing pubmed18n0824.xml.gz
writing pubmed18n0825.xml.gz
writing pubmed18n0826.xml.gz
writing pubmed18n0827.xml.gz
writing pubmed18n0828.xml.gz
writing pubmed18n0829.xml.gz
writing pubmed18n0830.xml.gz
writing pubmed18n0831.xml.gz
writing pubmed18n0832.xml.gz
writing pubmed18n0833.xml.gz
writing pubmed18n0834.xml.gz
writing pubmed18n0835.xml.gz
writing pubmed18n0836.xml.gz
writing pubmed18n0837.xml.gz
writing pubmed18n0838.xml.gz
writing pubmed18n0839.xml.gz
writing pubmed18n0840.xml.gz
writing pubmed18n0841.xml.gz
writing pubmed18n0842.xml.gz
writing pubmed18n0843.xml.gz
writing pubmed18n0844.xml.gz
writing pubmed18n0845.xml.gz
writing pubmed18n0846.xml.gz
writing pubmed

In [2]:
#Parse XML in each of the .gz files

filenames = glob.glob("*.gz")

for filename in filenames: 
    #parse csv as tree
    tree = ET.parse(gzip.open(filename))
    root = tree.getroot()

    # open a file for writing
    #print ("opening %s") % filename
    medline_data = open(filename + ".csv", 'w')

    # write column titles and variable names
    csvwriter = csv.writer(medline_data)
    
    medline_head = []

    journal = "Journal"
    medline_head.append(journal)
    year = "Year"
    medline_head.append(year)
    title = "Title"
    medline_head.append(title)
    fname_1st = "FName_1st"
    medline_head.append(fname_1st)
    lname_1st = "LName_1st"
    medline_head.append(lname_1st)
    fname_last = "FName_last"
    medline_head.append(fname_last)
    lname_last = "LName_last"
    medline_head.append(lname_last)
    affilication = "Affilication"
    medline_head.append(affilication)
    article_type = "Article_type"
    medline_head.append(article_type)
    middle_author = "Middle_author"
    medline_head.append(middle_author)
    has_abstract = "Abstract"
    medline_head.append(has_abstract)
    
    #write csv header
    csvwriter.writerow(medline_head)
    
    # If lines are empty, put in blanks
    def to_text (obj):
        if obj != None:
            return obj.text
        else:
            return ""
        
    # Extract Lines from Database
    for member in root.findall('PubmedArticle/MedlineCitation'):
        
            try:  
                journal = member.find("Article/Journal/Title")
                title = member.find("Article/ArticleTitle")
                year = member.find("Article/Journal/JournalIssue/PubDate/Year")
                article_type = member.find("Article/PublicationTypeList/PublicationType")
                authorlist = member.find("Article/AuthorList")
                
                authorlist = member.find("Article/AuthorList")
                
                if authorlist == None:
                    csvwriter.writerow([to_text(journal),to_text(year),to_text(title),"","","","","","",""])
                    continue

                authors = authorlist.findall("Author")

                fname_1st = authors[0].find("ForeName")
                lname_1st = authors[0].find("LastName")
                
                affilication = authors[0].find("AffiliationInfo/Affiliation")
                if  member.find("Article/Abstract") != None:
                    has_abstract=1
                else:
                    has_abstract=0

                if len(authors) == 1:            
                    fname_last = None
                    lname_last = None
                    middle_authors = None
                    
                if len(authors) == 2:            
                    fname_last = authors[-1].find("ForeName")
                    lname_last = authors[-1].find("LastName")
                    middle_authors = None
                    
                if len(authors) > 2:            
                    fname_last = authors[-1].find("ForeName")
                    lname_last = authors[-1].find("LastName")    
                    middle_author = authors[1:-1]
                    middle_authors = [to_text(auth.find("ForeName")) for auth in middle_author]
                

                csvwriter.writerow([to_text(journal),
                                    to_text(year),
                                    to_text(title), 
                                    to_text(fname_1st), 
                                    to_text(lname_1st), 
                                    to_text(fname_last), 
                                    to_text(lname_last),
                                    to_text(affilication),
                                    to_text(article_type),
                                    middle_authors,
                                    has_abstract])

            except Exception as e:
                print(e)
                #raise(e)
                continue
            
    print(filename + " Done")
    medline_data.close()


In [4]:
# CSV file concatonation
import glob, csv
from pandas import read_csv
theFiles=glob.glob('pubmed*.csv')
fout=open("Science.csv","w")

# get the headers (and the rest of the first file) 
print('processing {}'.format(theFiles[0]))
for line in open (theFiles[0]):
    if 'Science (New York, N.Y.)' in line:
        fout.write(line)
# get the rest of the files
for n in range(1,len(theFiles)):
    print('processing {}'.format(theFiles[n]))
    f=open(theFiles[n])
    for line in f:
        if 'Science (New York, N.Y.)' in line:
            fout.write(line)
    f.close()
fout.close()
print("Done")

df = read_csv('Science.csv')
df.columns = ["Journal","Year","Title","FName_1st","LName_1st","FName_last","LName_last","Affilication","Article_type","Middle_author","Abstract"]
df.to_csv('Science.csv')

processing /Users/yiqinshen/Downloads/ProcessedFilewithMidAuthor/NewData/pubmed18n0818.xml.gz.csv
processing /Users/yiqinshen/Downloads/ProcessedFilewithMidAuthor/NewData/pubmed18n0819.xml.gz.csv
processing /Users/yiqinshen/Downloads/ProcessedFilewithMidAuthor/NewData/pubmed18n0820.xml.gz.csv
processing /Users/yiqinshen/Downloads/ProcessedFilewithMidAuthor/NewData/pubmed18n0821.xml.gz.csv
processing /Users/yiqinshen/Downloads/ProcessedFilewithMidAuthor/NewData/pubmed18n0822.xml.gz.csv
processing /Users/yiqinshen/Downloads/ProcessedFilewithMidAuthor/NewData/pubmed18n0823.xml.gz.csv
processing /Users/yiqinshen/Downloads/ProcessedFilewithMidAuthor/NewData/pubmed18n0824.xml.gz.csv
processing /Users/yiqinshen/Downloads/ProcessedFilewithMidAuthor/NewData/pubmed18n0825.xml.gz.csv
processing /Users/yiqinshen/Downloads/ProcessedFilewithMidAuthor/NewData/pubmed18n0826.xml.gz.csv
processing /Users/yiqinshen/Downloads/ProcessedFilewithMidAuthor/NewData/pubmed18n0827.xml.gz.csv
processing /Users/yi

processing /Users/yiqinshen/Downloads/ProcessedFilewithMidAuthor/NewData/pubmed18n0909.xml.gz.csv
processing /Users/yiqinshen/Downloads/ProcessedFilewithMidAuthor/NewData/pubmed18n0910.xml.gz.csv
processing /Users/yiqinshen/Downloads/ProcessedFilewithMidAuthor/NewData/pubmed18n0911.xml.gz.csv
processing /Users/yiqinshen/Downloads/ProcessedFilewithMidAuthor/NewData/pubmed18n0912.xml.gz.csv
processing /Users/yiqinshen/Downloads/ProcessedFilewithMidAuthor/NewData/pubmed18n0913.xml.gz.csv
processing /Users/yiqinshen/Downloads/ProcessedFilewithMidAuthor/NewData/pubmed18n0914.xml.gz.csv
processing /Users/yiqinshen/Downloads/ProcessedFilewithMidAuthor/NewData/pubmed18n0915.xml.gz.csv
processing /Users/yiqinshen/Downloads/ProcessedFilewithMidAuthor/NewData/pubmed18n0916.xml.gz.csv
processing /Users/yiqinshen/Downloads/ProcessedFilewithMidAuthor/NewData/pubmed18n0917.xml.gz.csv
processing /Users/yiqinshen/Downloads/ProcessedFilewithMidAuthor/NewData/pubmed18n0918.xml.gz.csv
processing /Users/yi